### *Assignment 1*: User-based Collaborative Filtering Recommendations

In [1]:
from user_based_collaborative_filtering import UBCF
import pandas

**(a)** Download the MovieLens 100K rating dataset from https://grouplens.org/datasets/movielens/ 
(the small dataset recommended for education and development). Read the
dataset, display the first few rows to understand it, and display the count of ratings (rows)
in the dataset to be sure that you download it correctly

In [2]:
UBCF.ds.print_dataset_first_rows()

Max 5 csv rows displayed per file.

Display movies.csv
Number of elements:  9742
First elements:     movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


Display ratings.csv
Number of elements:  100836
First elements:     userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931




**(b)** Implement the user-based collaborative filtering approach, using the Pearson
correlation function for computing similarities between users, and

**(c)** the prediction function presented in class for predicting movies scores.

In [3]:
userA = 1
userB = 10
movieP = 1

In [4]:
sim = UBCF.sim_pcc(userA, userB)
print(f'Pearson correlation coefficient between user {userA} and {userB} is {sim}')

Pearson correlation coefficient between user 1 and 10 is -0.15121755406386428


$$sim^{PCC}(a, b) = \frac{\sum_{p \in P} (r_{a,p} - \bar{r_a})(r_{b,p} - \bar{r_b})}{\sqrt{\sum_{p \in P}(r_{a,p} - \bar{r_a})^2}\sqrt{\sum_{p \in P}(r_{b,p} - \bar{r_b})^2}}$$

In [5]:
neighbors = UBCF.top_n_similar_users(30)
prediction = UBCF.prediction_from_neighbors(userA, movieP, neighbors)
print(f'Prediction for the rating of movie {movieP} rating by user {userA} is {prediction}')

Prediction for the rating of movie 1 rating by user 1 is 4.5574254096356075


$$pred(a,p)=\bar{r_a} + \frac{\sum_{b \in N}sim(a,b)\cdot(r_{b,p}-\bar{r_b})}{\sum_{b \in N}sim(a,b)}$$

**(d)** Select a user from the dataset, and for this user, show the 10 most similar users and
the 10 most relevant movies that the recommender suggests.

In [6]:
similar_users = UBCF.top_n_similar_users(userA, n=10)
sim_users_df = pandas.DataFrame(similar_users, columns=['userId', 'similarity'])

print(sim_users_df.to_string(index=False))

 userId  similarity
     77    1.000000
     12    1.000000
     85    1.000000
    253    1.000000
    291    1.000000
    358    1.000000
    388    1.000000
      2    1.000000
    146    0.999050
    278    0.971061


In [7]:
top_rec = UBCF.top_n_recommendations(userA)
top_rec_df = pandas.DataFrame(top_rec, columns=['movieId', 'prediction'])

print(top_rec_df.to_string(index=False))

 movieId  prediction
     319    6.769157
    3567    6.726379
     555    6.641379
     913    6.252743
   55276    6.252743
   30803    6.225754
    3972    6.223522
   27611    6.223522
    5066    6.110282
   42728    6.110282
